In [145]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [146]:
# i added skipinitialspace becuase it returns null when i created title column because of the whitespace
train = pd.read_csv('train.csv', skipinitialspace = True)
test = pd.read_csv('test.csv', skipinitialspace = True)

In [147]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [148]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [149]:
train.shape

(891, 12)

In [150]:
test.shape

(418, 11)

In [151]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [152]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [153]:
train_test = [train,test]

In [154]:
# extracing the first titles of the names into new column called title
for data in train_test:
    data['title'] = data['Name'].str.extract('([A-Za-z]+)\.', expand =False)

In [155]:
train['title'].value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Major         2
Col           2
Mlle          2
Ms            1
Countess      1
Don           1
Sir           1
Jonkheer      1
Mme           1
Lady          1
Capt          1
Name: title, dtype: int64

In [156]:
train['title'].dtypes

dtype('O')

In [157]:
# chainging the values into numerical values 
title_map = {'Mr':0,'Miss':1,'Mrs':2,'Master':3,'Dr':3,'Rev':3,'Col':3,'Ms':3,'Mlle':3,
                            'Major':3,'Dona':3,'Lady':3,'Don':3,'Countess':3,'Capt':3,'Mme':3,'Sir':3,
                            'Jonkheer':3}
for data in train_test:
    data['title'] = data['title'].map(title_map)

In [158]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [159]:
train.drop(columns = 'Name', axis =1, inplace = True, )

In [160]:
test.drop(columns = 'Name', axis =1, inplace = True)

In [161]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
0,892,3,male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,female,47.0,1,0,363272,7.0000,NaN,S,2
2,894,2,male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,female,22.0,1,1,3101298,12.2875,NaN,S,2


### explanation 

In [162]:
# change the sex column values into 0 for males and 1 for females 
sex_mapping = {'male':0, 'female':1}
for i in train_test:
    i['Sex'] = i['Sex'].map(sex_mapping)

In [163]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
0,1,0,3,0,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,1,38.0,1,0,PC 17599,71.2833,C85,C,2
2,3,1,3,1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,1,35.0,1,0,113803,53.1000,C123,S,2
4,5,0,3,0,35.0,0,0,373450,8.0500,NaN,S,0


In [164]:
# check for missing values 
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
title            0
dtype: int64

In [165]:
train.Age.mean()

29.69911764705882

In [166]:
# fill in the missing values of the age column with the median 
train['Age'].fillna(value = 29.69, inplace = True)
test['Age'].fillna(value = 29.69, inplace = True)
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
0,1,0,3,0,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,1,38.0,1,0,PC 17599,71.2833,C85,C,2
2,3,1,3,1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,1,35.0,1,0,113803,53.1000,C123,S,2
4,5,0,3,0,35.0,0,0,373450,8.0500,NaN,S,0


In [167]:
# get the median of the values in the embarked column to fill inn missing Values
train.Embarked.median

<bound method Series.median of 0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object>

In [168]:
# fill in the missing values with the median S
train.Embarked.fillna(value = 'S', inplace = True)

In [169]:
embarked_map = {'S':0, 'C':1,'Q':2}
for i in train_test:
    i['Embarked'] = i['Embarked'].map(embarked_map)

In [170]:
for i in train_test:
    i.loc[i['Fare']<=17, 'Fare'] =0,
    i.loc[(i['Fare']>17)& (i['Fare']<=30),'Fare'] = 1,
    i.loc[(i['Fare']>30)& (i['Fare']<=100),'Fare'] = 2,
    i.loc[i['Fare']>17, 'Fare'] = 3

In [171]:
train.Embarked.value_counts()

0    646
1    168
2     77
Name: Embarked, dtype: int64

In [172]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
0,1,0,3,0,22.0,1,0,A/5 21171,0.0,NaN,0,0
1,2,1,1,1,38.0,1,0,PC 17599,2.0,C85,1,2
2,3,1,3,1,26.0,0,0,STON/O2. 3101282,0.0,NaN,0,1
3,4,1,1,1,35.0,1,0,113803,2.0,C123,0,2
4,5,0,3,0,35.0,0,0,373450,0.0,NaN,0,0


In [173]:
# extract the first chcaracter of the cabin number 
for i in train_test:
    i['Cabin'] = i['Cabin'].str[:1]

In [174]:
train.Cabin.unique()

array([nan, 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [175]:
cabin_map = {'B':0, 'C':1, 'E':2, 'D':3, 'A':4, 'T':5, 'F':6, 'G':7}
for i in train_test:
    i['Cabin'] = i['Cabin'].map(cabin_map)


In [176]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
0,1,0,3,0,22.0,1,0,A/5 21171,0.0,NaN,0,0
1,2,1,1,1,38.0,1,0,PC 17599,2.0,1.0,1,2
2,3,1,3,1,26.0,0,0,STON/O2. 3101282,0.0,NaN,0,1
3,4,1,1,1,35.0,1,0,113803,2.0,1.0,0,2
4,5,0,3,0,35.0,0,0,373450,0.0,NaN,0,0


In [177]:
train.Cabin.fillna(train.groupby('Pclass')['Cabin'].transform('median'),
                   inplace = True)
test.Cabin.fillna(test.groupby('Pclass')['Cabin'].transform('median'), 
                  inplace = True)


In [178]:
train.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
title          0
dtype: int64

In [179]:
# we need to dropp the ticket column 
train.drop(columns = 'Ticket', inplace = True)
test.drop(columns = 'Ticket', inplace = True)

In [180]:
train.drop(columns = 'PassengerId', inplace = True)
test.drop(columns = 'PassengerId').copy()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,title
0,3,0,34.50,0,0,0.0,6.0,2,0
1,3,1,47.00,1,0,0.0,6.0,0,2
2,2,0,62.00,0,0,0.0,6.0,2,0
3,3,0,27.00,0,0,0.0,6.0,0,0
4,3,1,22.00,1,1,0.0,6.0,0,2
...,...,...,...,...,...,...,...,...,...
413,3,0,29.69,0,0,0.0,6.0,0,0
414,1,1,39.00,0,0,3.0,1.0,1,3
415,3,0,38.50,0,0,0.0,6.0,0,0
416,3,0,29.69,0,0,0.0,6.0,0,0


In [181]:
target = train['Survived']

In [182]:
#drop the survive column from the training set 

In [183]:
train.drop(columns = 'Survived', inplace = True)

In [184]:
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,title
0,3,0,22.0,1,0,0.0,6.0,0,0
1,1,1,38.0,1,0,2.0,1.0,1,2
2,3,1,26.0,0,0,0.0,6.0,0,1
3,1,1,35.0,1,0,2.0,1.0,0,2
4,3,0,35.0,0,0,0.0,6.0,0,0


In [185]:
target.shape, train.shape

((891,), (891, 9))

In [186]:
# the number of columns in the train dataset is 9 and the numner od the columns in the test dataset is 20 
# we have tpo make them match
test_data = test.drop('PassengerId',axis = 1).copy()

In [187]:
# choosing the KFold metrics
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 2)

In [188]:
Knn = KNeighborsClassifier(n_neighbors = 5)
Knn.fit(train,target)
y_pred = Knn.predict(test_data)
score = cross_val_score(Knn,train,target,cv = k_fold, scoring = 'accuracy')
print(score)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [137]:
# using DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train, target)
y_pred = clf.predict(test_data)
score = cross_val_score(clf,train,target,cv = k_fold, scoring = 'accuracy')
print(score)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [189]:
# using RandomForestClassifier
Rfc = RandomForestClassifier(n_estimators = 12)
Rfc.fit(train,target)
y_pred = Rfc.predict(test_data)
score = cross_val_score(Rfc, train, target, cv = k_fold, scoring = 'accuracy')
print(score)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [139]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(train,target)
prediction = clf.predict(test_data)
score = cross_val_score(clf, train,target, cv = k_fold , scoring = 'accuracy')
print(score)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [190]:
csv_file = pd.DataFrame({'PassengerId':test['PassengerId'],
                        'Survived':prediction})
csv_file.to_csv('Titanic_submission.csv', index = False)

NameError: name 'prediction' is not defined

In [141]:
# becuase it returns value error ( there is an infinite number or nan). I checked them all and found Fare column has infinite
np.isfinite(test).sum()


PassengerId    418
Pclass         418
Sex            418
Age            418
SibSp          418
Parch          418
Fare           417
Cabin          418
Embarked       418
title          418
dtype: int64

In [142]:
# replacing the infinite value in the dataset 
test['Fare'].replace([np.inf,2], inplace = True)

In [143]:
# to chedck if there is an infinite value in the dataset 
np.isfinite(test).sum()

PassengerId    418
Pclass         418
Sex            418
Age            418
SibSp          418
Parch          418
Fare           417
Cabin          418
Embarked       418
title          418
dtype: int64

In [144]:
test['Fare'].fillna(value= 2, inplace = True)